In [ ]:
import import_ipynb
import torch
import gpytorch 
import math
import copy 
from buildModel import ExactGPmodel


In [ ]:
def optimization(llh,model,Xtrain,ytrain,tolerance_grad_adpt,**kwargs):
    """
    Performs optimization on a Gaussian Process model using the LBFGS optimizer.
    NOTE: LBFGS generally performs better than ADAM, especially for datasets not too large (<2000 samples).
    Args:
        llh (Likelihood): The likelihood component of the GP model.
        model (GPModel): The Gaussian Process model.
        Xtrain (Tensor): Training input data.
        ytrain (Tensor): Training target data.
        tolerance_grad_adpt (float): Tolerance for gradient convergence.
        **kwargs: Additional keyword arguments for customizing optimization settings.
    
    Returns:
        Tensor: Optimized weights or parameters as a tensor.
    """
    # Retrieve optimization parameters from kwargs with defaults
    learning_rate = kwargs.get('learning_rate',0.01)
    training_iter = kwargs.get('training_iter',200)
    ifPrint = kwargs.get('ifPrint',True)
    numPrint = kwargs.get('numPrint',100)
    logScale = kwargs.get('logScale',True)

    # Define the LBFGS optimizer with specified learning rate and tolerance
    optimizer = torch.optim.LBFGS(model.parameters(), history_size=15,line_search_fn='strong_wolfe',lr = learning_rate,tolerance_grad=tolerance_grad_adpt)
    
    # Marginal log likelihood for Gaussian Processes
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(llh,model)

     # Optimization loop
    for i in range(training_iter):
        def closure():
            optimizer.zero_grad()
            output = model(Xtrain)
            loss = -mll(output, ytrain)
            loss.backward()
            return loss
        # Perform a single optimization step
        loss = optimizer.step(closure)
        if ifPrint is True:
            if (i+1) % numPrint == 0:
                log_message = 'Iter %d/%d LOSS: %.3f ' % (i + 1, training_iter, loss.item())
                lengthscale_values = model.covar_module.base_kernel.lengthscale[0]   
                for idx, lengthscale in enumerate(lengthscale_values):
                    if logScale is True:
                        log_message += ' W%d: %.3f' % (idx + 1, math.log(1 / lengthscale.item()))
                    else:
                        log_message += ' W%d: %.2e' % (idx + 1, 1 / lengthscale.item())
                noise_value = model.likelihood.noise.item()
                log_message += ' noise: %.4f' % noise_value
                print(log_message)
    if logScale is True:
        w = torch.log(1/model.covar_module.base_kernel.lengthscale[0])
    else:
        w = 1/model.covar_module.base_kernel.lengthscale[0]
    return w

In [ ]:

def Inference(Xtrain,Ytrain,**kwargs):
    """
    Performs a series of optimizations on a Gaussian Process model for each dimension (each node) of 
    the target variable Ytrain using training data Xtrain.

    Args:
        Xtrain (Tensor): Input features for the training data (T,N\times\Lambda).
        Ytrain (Tensor): Target variables for the training data(T,N).
        **kwargs: Keyword arguments for additional parameters like gradient tolerance.
        **kwargs are store in param.ipynb

    Returns:
        Tensor: A matrix W representing the weighed adjacency matrix, each row represents the contribution from all other nodes
        to a specific node.
    """
    # Determine the number of target dimensions in Ytrain
    N = Ytrain.size()[1]
    # Extract the number of data points and features from Xtrain
    [numData,numFeatures] = Xtrain.size()
    # Initialize the matrix to store optimized weights
    W = torch.zeros(N,numFeatures) 
    # Set initial values for the model's parameters
    ini = 10*torch.ones([1,numFeatures])

    # Process each node 
    for n in range(N):
        print("Processing for nodes " + str(n+1))
        # Retrieve or set default gradient tolerance from kwargs
        tolerance_grad_adpt =  kwargs.get('tolerance_grad',1e-6)
        #  Extract the target variable for the current dimension
        ytrain = Ytrain[:,n]
        
        # Attempt to optimize until successful or tolerance limit is reached
        while True:
            try:
                # Initialize the likelihood with a noise constraint
                llh = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.GreaterThan(1e-4))
                # Set the initial noise based on the standard deviation of ytrain
                llh.noise_covar.noise = ytrain.std()
                
                # Create and configure the Gaussian Process model
                model = ExactGPmodel(Xtrain, ytrain, llh)       
                model.covar_module.base_kernel.lengthscale = ini
                model.covar_module.outputscale = ytrain.std()
        
                # Set model and likelihood to training mode
                model.train()
                llh.train()
                
                 # Perform optimization
                w = optimization(llh,model,Xtrain,ytrain,tolerance_grad_adpt,**kwargs)
                # Break the loop upon successful optimization
                break
            except Exception as e:
                # Log the error and adjust the tolerance if under the threshold
                print("--------------------------------------------------*******----------------------------------------------------------")
                print(f"Error during optimization: {e}")
                if tolerance_grad_adpt<1:
                    tolerance_grad_adpt *= 10
                    print("Tolerance of gradient increased to "+ str(tolerance_grad_adpt))
                    print("---------------------------------------------------------------------------------------------------------------")
                else:
                    # If tolerance exceeds the threshold, log failure and exit loop
                    print("Failed for optimization after increasing the tolerance of gradient")
                    break
         # Store the optimized weights for the current dimension
        W[n,:] = w
    # Return the matrix of optimized weights
    return W